In [0]:
!pip3 install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 49kB/s 
     |████████████████████████████████| 450kB 53.3MB/s 
     |████████████████████████████████| 3.8MB 48.9MB/s 
     |████████████████████████████████| 81kB 14.3MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.8.2 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled goog

In [0]:
import os

import tensorflow as tf
import numpy as np


In [0]:
from tensorflow.keras.applications.VGG16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_img_data_gen = ImageDataGenerator(shear_range=0.5, rotation_range=50, zoom_range=0.2, 
                                        width_shift_range=0.2, height_shift_range=0.2, fill_mode='constant',rescale=1./255
                                        preprocessing_function=preprocess_input
)
    
train_mask_data_gen = ImageDataGenerator(shear_range=0.5, rotation_range=50, zoom_range=0.2,
                                        width_shift_range=0.2, height_shift_range=0.2, fill_mode='constant')

valid_img_data_gen = ImageDataGenerator(shear_range=0.5, rotation_range=50, zoom_range=0.2, 
                                        width_shift_range=0.2, height_shift_range=0.2, fill_mode='constant',rescale=1./255)

valid_mask_data_gen = ImageDataGenerator(shear_range=0.5, rotation_range=50, zoom_range=0.2,
                                        width_shift_range=0.2, height_shift_range=0.2, fill_mode='constant')

test_img_data_gen = ImageDataGenerator(rescale=1./255)
test_mask_data_gen = ImageDataGenerator()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd "/content/drive/My Drive/Colab Notebooks"


/content/drive/My Drive/Colab Notebooks


In [0]:
dataset_dir = os.path.join("/content/drive/My Drive/Colab Notebooks", 'Segmentation_Dataset')

# Batch size
bs = 8

# img shape
img_h = 256
img_w = 256

num_classes=1

# Training

training_dir = os.path.join(dataset_dir, 'training')
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                      seed=SEED)  

train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         color_mode='grayscale',
                                                         batch_size=bs,
                                                         class_mode=None, 
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         seed=SEED)


label_generator = (tf.cast(x, tf.int32) for x in train_mask_gen)
train_gen = zip(train_img_gen, label_generator)


# Validation
validation_dir = os.path.join(dataset_dir, 'validation')
valid_img_gen = valid_img_data_gen.flow_from_directory(os.path.join(validation_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, 
                                                       shuffle=False,
                                                       interpolation='bilinear',
                                                       seed=SEED)
valid_mask_gen = valid_mask_data_gen.flow_from_directory(os.path.join(validation_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs, 
                                                         class_mode=None, 
                                                         shuffle=False,
                                                         interpolation='bilinear',
                                                         seed=SEED)
valid_gen = zip(valid_img_gen, valid_mask_gen)


# Test
test_dir = os.path.join(dataset_dir, 'test')
test_img_gen = test_img_data_gen.flow_from_directory(os.path.join(test_dir, 'images'),
                                                     target_size=(img_h, img_w),
                                                     batch_size=2, 
                                                     class_mode=None, # Because we have no class subfolders in this case
                                                     shuffle=False,
                                                     interpolation='bilinear',
                                                     seed=SEED)



Found 7640 images belonging to 1 classes.
Found 7640 images belonging to 1 classes.
Found 1234 images belonging to 1 classes.


"\ntest_mask_gen = test_mask_data_gen.flow_from_directory(os.path.join(test_dir, 'masks'),\n                                                       target_size=(img_h, img_w),\n                                                       batch_size=bs, \n                                                       class_mode=None, # Because we have no class subfolders in this case\n                                                       shuffle=False,\n                                                       interpolation='bilinear',\n                                                       seed=SEED)\ntest_gen = zip(test_img_gen, test_mask_gen)\n"

In [0]:


# Training

train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w,1]))

def prepare_target(img, mask):
    mask = tf.cast(tf.where(mask > 0.5, x=1, y=0), tf.int32)
    return (img, mask)

train_dataset = train_dataset.map(prepare_target)


train_dataset = train_dataset.repeat()



valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 3]))
valid_dataset = valid_dataset.map(prepare_target)


valid_dataset = valid_dataset.repeat()


test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
test_dataset = test_dataset.map(prepare_target)

test_dataset = test_dataset.repeat()

# Build U-Net model

In [0]:

inputs = tf.keras.layers.Input((256, 256, 3))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(s)
#c1 = tf.keras.layers.BatchNormalization()(c1)                            
c1 = tf.keras.layers.Dropout(0.1)(c1)

c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c1)
#c1 = tf.keras.layers.BatchNormalization()(c1)                            
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p1)
#c2 = tf.keras.layers.BatchNormalization()(c2)                            

c2 = tf.keras.layers.Dropout(0.2)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c2)

#c2 = tf.keras.layers.BatchNormalization()(c2)                            
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p2)

#c3 = tf.keras.layers.BatchNormalization()(c3)                            
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c3)
#c3 = tf.keras.layers.BatchNormalization()(c3)                              
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p3)
#c4 = tf.keras.layers.BatchNormalization()(c4)                              
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c4)
#c4 = tf.keras.layers.BatchNormalization()(c4)                            
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p4)
#c5 = tf.keras.layers.BatchNormalization()(c5)                            
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c5)
#c5 = tf.keras.layers.BatchNormalization()(c5)

u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u6)
#c6 = tf.keras.layers.BatchNormalization()(c6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c6)
#c6 = tf.keras.layers.BatchNormalization()(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u7)
#c7 = tf.keras.layers.BatchNormalization()(c7)                          
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c7)
#c7 = tf.keras.layers.BatchNormalization()(c7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u8)
#c8 = tf.keras.layers.BatchNormalization()(c8)                            
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c8)
#c8 = tf.keras.layers.BatchNormalization()(c8) 

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u9)
#c9 = tf.keras.layers.BatchNormalization()(c9)                             
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c9)
#c9 = tf.keras.layers.BatchNormalization()(c9)                              
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)


model = tf.keras.Model(inputs=[inputs], outputs=[outputs])


In [0]:
def my_IoU(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32) 
    intersection = tf.reduce_sum(y_true * y_pred)
   
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return intersection / union

Loss

In [0]:
lr = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

metrics=[my_IoU]

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         lambda[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 256, 256, 16) 0           conv2d[0][0]                     
______________________________________________________________________________________________

In [0]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

lrr = ReduceLROnPlateau(monitor='my_IoU', factor=0.1, patience=3, min_lr=0.00001, verbose=1)

callbacks = [lrr]

segment = model.fit(x=train_dataset,
          epochs=100,  
          steps_per_epoch=len(train_img_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen),
          callbacks=callbacks
)


Train for 955 steps
Epoch 1/50
185/955 [====>.........................] - ETA: 2:10:53 - loss: 0.5843 - my_IoU: 0.2224

In [0]:
def rle_encode(img):
      # Flatten column-wise
      pixels = img.T.flatten()
      pixels = np.concatenate([[0], pixels, [0]])
      runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
      runs[1::2] -= runs[::2]
      return ' '.join(str(x) for x in runs)

In [0]:

img_filename = next(os.walk("/content/drive/My Drive/Colab Notebooks/Segmentation_Dataset/test/images/img"))[2]

In [0]:
import os
from datetime import datetime

def create_csv(results):

    
    csv_fname = '/content/drive/My Drive/Colab Notebooks/Segmentation_Dataset/file.csv'

    with open(csv_fname, 'w+') as f:
      print("true")
      f.write('ImageId,EncodedPixels,Width,Height\n')

      for key, value in results.items():
   
          f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')

In [0]:
results ={}
from PIL import Image
for img_file in img_filename:
  path = "/content/drive/My Drive/Colab Notebooks/Segmentation_Dataset/test/images/img/" + img_file
  img = Image.open(path)
  img = img.resize((256,256))
  img_arr = np.expand_dims(np.array(img),0)
  prediction_mask = model.predict(x = img_arr/255.)
  prediction_mask = (prediction_mask > 0.5)
  mask = prediction_mask[0]
  mask_pred = rle_encode(prediction_mask)
  name = os.path.splitext(img_file)[0]
  
  results[name] = mask_pred  

create_csv(results)

true
